In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from pprint import pprint
import csv
import random
import requests, re, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
def datareader(clubname, season):
    data = pd.read_csv('transfers.csv')
    dataPos = pd.read_csv('game_lineups.csv')
    # recentData = data[data['transfer_season'] == '18/19']
    seasonIn = data[data['transfer_season'] == season]
    #seasonID = seasonIn['to_club_id'].iloc[0]

    clubIn = seasonIn[seasonIn['to_club_name'] == clubname]
    clubID = clubIn['to_club_id'].iloc[0]
    
    clubPos = dataPos[dataPos['club_id'] == clubID]
    mergedClub = pd.merge(clubIn, clubPos, on='player_id', how = 'inner')
    
    '''
    #starting_year = str(start_year) + '-06-01';
    #ending_year = str(start_year+1) + '-06-01';
    starting_year = '2018-08-01';
    ending_year = '2019-08-01';
    #starting_year = datetime.strptime(str(start_year) + '-06-01', '%Y-%m-%d').date()
    #ending_year = datetime.strptime(str(start_year+1) + '-06-01', '%Y-%m-%d').date()
    #mergedClub = mergedClub[datetime.strptime(mergedClub['date'], '%Y-%m-%d').date() >= starting_year]
    #mergedClub = mergedClub[datetime.strptime(mergedClub['date'], '%Y-%m-%d').date() <= ending_year]
    '''
    #mergedClub = mergedClub[mergedClub['date'] >= starting_year]
    #mergedClub = mergedClub[mergedClub['date'] <= ending_year]
    mergedClub['type'] = np.where(mergedClub['type'] == 'starting_lineup', 1, 0)
    pos = {}
    for i in mergedClub.index:
        if mergedClub['player_id'][i] in pos:
            if mergedClub['position'][i] not in pos[mergedClub['player_id'][i]]:
                pos[mergedClub['player_id'][i]].append(mergedClub['position'][i])
        else:
            pos[mergedClub['player_id'][i]] = [mergedClub['position'][i]]
    newDF = mergedClub.groupby('player_id').agg(total = ('type','count'), starts =('type','sum'),
        fee = ('transfer_fee', 'mean'), value = ('market_value_in_eur', 'mean'), playerId = ('player_id','mean'))
    newDF['position'] = newDF['playerId'].map(pos)
    newDF['start%'] = (newDF['starts']/newDF['total']).round(4) * 100
    return newDF

In [3]:
dataf = datareader('Wolves', '21/22')
print(dataf)

           total  starts         fee       value  playerId  \
player_id                                                    
204103       202     107         0.0  20000000.0  204103.0   
249994        93      87   8000000.0   7500000.0  249994.0   
265078        35       8         0.0   3500000.0  265078.0   
292246        69      38         0.0  11000000.0  292246.0   
357158        97      26         0.0   5000000.0  357158.0   
412669        38      16         0.0  22000000.0  412669.0   
415646        22       0         0.0   1600000.0  415646.0   
429014       111      23         0.0  11000000.0  429014.0   
444641        20       0         0.0         NaN  444641.0   
504581        18       2         0.0   2500000.0  504581.0   
532541        48       2         NaN         NaN  532541.0   
578391       113      62  11100000.0  18000000.0  578391.0   
606718        62      28         0.0   1000000.0  606718.0   
695454        16       1   3500000.0   1000000.0  695454.0   
716276  

In [4]:
def tableposition(clubname):
    table_data = pd.read_csv('games.csv')
    
    
    

In [5]:
data = datareader('Fulham')
print(data)

TypeError: datareader() missing 1 required positional argument: 'start_year'

In [6]:
dataf = datareader('Cardiff', 603)
print(dataf)

           total  starts         fee      value  playerId  \
player_id                                                   
55742         27      24         0.0  7000000.0   55742.0   
126014        14       4   3450000.0  2000000.0  126014.0   
186186        39      17  11350000.0  3000000.0  186186.0   

                                                    position     start%  
player_id                                                                
55742      [Central Midfield, Left Midfield, Defensive Mi...  88.888889  
126014     [Central Midfield, Right Midfield, Left-Back, ...  28.571429  
186186     [Centre-Forward, Left Midfield, Central Midfie...  43.589744  


In [9]:
dataf = datareader('Cardiff', 603)
dataf.corr(numeric_only = True)

,total,starts,fee,value,playerId,start%
total,1.000000,0.658025,0.661727,0.211604,0.440213,0.261499
starts,0.658025,1.000000,-0.129123,0.875186,-0.386440,0.898868
fee,0.661727,-0.129123,1.000000,-0.592744,0.964492,-0.550616
value,0.211604,0.875186,-0.592744,1.000000,-0.784411,0.998681
playerId,0.440213,-0.386440,0.964492,-0.784411,1.000000,-0.751535
start%,0.261499,0.898868,-0.550616,0.998681,-0.751535,1.000000
